In [1]:
%pip install -qU langchain
%pip install -qU huggingface_hub
%pip install nest-asyncio
%pip install unstructured
%pip install chromadb

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
# Necessary to import OpenAI
with open('assets/openai_api_key', 'r') as f:
    openai_api_key = f.read()
with open('assets/huggingface_api_key', 'r') as f:
    huggingface_api_key = f.read()
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['HUGGINGFACEHUB_API_KEY'] = huggingface_api_key
from pathlib import Path
import nest_asyncio
nest_asyncio.apply()
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.document_loaders import DirectoryLoader
from langchain.output_parsers import RegexParser
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
import json
import pandas as pd
import re

# OpenAI QA

We use Langchain to create a Chroma database of our papers against which we will query our questions. The answers are provided in the requested structure, making it straightforward to extract information.

Every paper is evaluated separately.

TODO: improve documentation

https://python.langchain.com/docs/modules/chains/additional/question_answering.html

https://python.langchain.com/docs/modules/chains/additional/openai_functions_retrieval_qa


Setting up the parameters:

In [3]:
paper_path = '../data/docs/'
pickle_path = '../data/pickle/'
chroma_path = '../data/chroma/'
MODEL = 'gpt-3.5-turbo'

Creating the vector database:

In [4]:
def papers_to_vectors(paper_path = paper_path, chroma_path=chroma_path):
    #TODO chroma_index = chroma_path + 'index'
    loader = DirectoryLoader(paper_path)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    embeddings = OpenAIEmbeddings()
    docsearch = Chroma.from_documents(texts, embeddings,persist_directory='../data/chroma', metadatas=[{"source": str(i)} for i in range(len(texts))])
    docsearch.persist()
    return docsearch

docsearch = papers_to_vectors()



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Defining the prompt template:

In [3]:
prompt_template = """We are trying to extract information from a journal article where they describe assays on graphene oxide to find its effects on biological systems and organisms. We want to extract information about the assays they performed, and how different measurement groups and conditions lead or not to different effects.  
Use the journal article as context to fill out the keys to return as defined at the end. You will provide an answer with the format below for the source document I have provided. If you don't know the answer, reply with a blank string (''), do not try to make up an answer. If you find several instances of an adverse outcome, please return each key answer as a comma-separated list in such a way that e.g. element 1 of a given key (e.g., diameter measurement units) corresponds with element 1 from a different key (e.g. diameter measurement value) I provide an explanation between brackets of what each key to return is, but do not include those in your answer, just the key name exactly as provided followed by a colon and your answer. 

Some examples of adverse outcomes you should look for are: apoptosis, ROS formation or oxidative stress, cell membrane disruption, cell viability reduction, mortality, cell viability, DNA damage, cytotoxicity, pathway activation, accumulation...

In addition to answering to the specific keys, also return a 'Score' of how fully it answered the question, the journal article used as context, and the text passage containing the answer. All these keys have to be in the following #format, do not alter the order or skip any key:

Format:
---------
Question: [question here] (for all questions posed)
Score: [score between 0 and 100]
Source: [source, the journal article containing the answer, from documents metadata]
Passage: [the text passage containing the answer]


---------
Example answer (keys might differ):
Adverse outcome:
Bioassay:
Material diameter measurement:
 "Numerical value", "Units

---------
Context:

{context}
---------
Questions: 
{question}

"""



In [6]:
documents = os.listdir(paper_path)
print("- "+ "\n- ".join(documents))


- j.toxlet.2010.11.016.xml
- C2JM31396K.pdf
- nn101097v.pdf
- j.biomaterials.2013.01.001.xml
- acsomega.2c03171.xml
- tx400385x.pdf
- j.biomaterials.2012.07.040.xml
- j.biomaterials.2012.05.064.xml
- smll.201201546.pdf
- nn202699t.pdf
- nn1007176.pdf
- A20060216.pdf
- s11671-010-9751-6.xml
- j.envpol.2017.12.034.xml
- cdd.2010.11.pdf
- nl202515a.pdf
- smll.201102743.pdf
- j.biomaterials.2012.02.021.xml
- j.biomaterials.2012.11.001.xml
- la203607w.pdf
- am300253c.pdf
- ijms221910578.xml


All together?

In [ ]:
prompt_template = """We are trying to extract information from a journal article where they describe assays on graphene oxide to find its effects on biological systems and organisms. We want to extract information about the assays they performed, and how different measurement groups and conditions lead or not to different effects.  
Use the journal article as context to fill out the keys to return as defined at the end. You will provide an answer with the format below for the source document I have provided. If you don't know the answer, reply with a blank string (''), do not try to make up an answer. If you find several instances of an adverse outcome, please return each key answer as a comma-separated list in such a way that e.g. element 1 of a given key (e.g., diameter measurement units) corresponds with element 1 from a different key (e.g. diameter measurement value) I provide an explanation between brackets of what each key to return is, but do not include those in your answer, just the key name exactly as provided followed by a colon and your answer. 

Some examples of adverse outcomes you should look for are: apoptosis, ROS formation or oxidative stress, cell membrane disruption, cell viability reduction, mortality, cell viability, DNA damage, cytotoxicity, pathway activation, accumulation...

In addition to answering to the specific keys, also return a 'Score' of how fully it answered the question, the journal article used as context, and the text passage containing the answer. All these keys have to be in the following #format, do not alter the order or skip any key:

Format:
---------
Question: [question here] (for all questions posed)
Score: [score between 0 and 100]
Source: [source, the journal article containing the answer, from documents metadata]
Passage: [the text passage containing the answer]


---------
Example answer (keys will probably be different):
Adverse outcomes: cell viability decrease, none, ...
Bioassays: MTT, MTT, ...
Coatings: none, PEG, ... 
(...)

---------
Context:

{context}
---------
Questions: 
{question}

"""



In [ ]:
all = """
    Adverse outcomes (The adverse outcomes?):
    Bioassays (What specific bioassays were performed?):
    Graphene oxide dose (What was the graphene oxide dose that produced this  outcome?):
    Graphene oxide dose units (What are the units of the graphene oxide dose that produced outcome i?):
    Coating (What type of nanomaterial coating is reported?):
    Organism/Biological system (What organism or biological system was the assay performed on (include strain, if applicable)?):
    Hazardous shape (What is the shape of the graphene oxide that produced this outcome?):
    Hazardous diameter (What is the diameter of the graphene oxide that produced this outcome?): 
    Non-hazardous shape (What is the shape of the graphene oxide that did not produce this outcome?):
    Non-hazardous diameter (What is the diameter of the graphene oxide that did not produce this outcome?):
    Hazardous time points (At which time points did graphene oxide produce the adverse effect?):
    Non-hazardous time points (At which time points did graphene oxide not produce the adverse effect?):"""

output_keys=["Adverse outcomes","Bioassays", "Hazardous doses", "Hazardous doses units", "Coatings", "Organisms/Biological systems", "Hazardous shapes", "Hazardous diameters", "Non-hazardous shapes", "Non-hazardous diameters" , "Hazardous time points", "Non-hazardous time points", "Score", "Source", "Passage"]

results = {}
for document in documents:
    query = f"Select the document with name: {document}"
    docs = docsearch.as_retriever().get_relevant_documents(query)
    chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff",    prompt=doc_prompt)
    
    res = chain({"input_documents": docs, "question": all},    return_only_outputs=False)
    output = res['output_text'].split("\n")
    results[document] = output
    
output_parser = RegexParser(
    regex=r"\nBioassay: (.*)\nMaterial diameter measurement: (.*)\nNumerical value: (.*)\nUnits: (.*)\nGraphene oxide dose: (.*)\nGraphene oxide dose units: (.*)\nOrganism/Biological system: (.*)\nScore: (.*)\nSource: (.*)\nPassage: (.*)",
    output_keys=output_keys,
)

doc_prompt = PromptTemplate(
    template=prompt_template,
    input_variables=['context', 'question'],
    output_parser=output_parser,
)

all = {key:{} for key in results}
for key in results:
    for output in output_keys:
        for value in results[key]:
            if output in value:
                new_val = value.split(output)[1]
                if ':' in new_val:
                    new_val = new_val.replace(":", "")
                    all[key][output] = new_val
df_all = pd.DataFrame(all) 

## Diameter

Extracting adverse outcomes related to diameter from each paper:

In [7]:
diameter = """
    Adverse outcomes (What were the adverse outcome?):
    Bioassay (What specific bioassay(s) were performed?):
    Material diameter measurement (What type of material diameter measurement is reported?):
    Numerical value (What is the numerical value of the diameter measurement?):
    Units (What are the units of the diameter measurement, if applicable?):
    Graphene oxide dose (What was the graphene oxide dose that produced this  outcome?):
    Graphene oxide dose units (What are the units of the graphene oxide dose that produced this  outcome?):
    Organism/Biological system (What organism or biological system was the assay performed on (include strain, if applicable)?"""

In [8]:
results = {}

#TODO 
output_parser = RegexParser(
    regex=r"Adverse outcome: (.*)\n\nBioassay: (.*)\nMaterial diameter measurement: (.*)\nNumerical value: (.*)\nUnits: (.*)\nGraphene oxide dose: (.*)\nGraphene oxide dose units: (.*)\nOrganism/Biological system: (.*)\nScore: (.*)\nSource: (.*)\nPassage: (.*)",
    output_keys=["Adverse outcome","Bioassay", "Material diameter measurement", "Numerical value", "Units", "Graphene oxide dose", "Graphene oxide dose units", "Organism/Biological system", "score", "Source", "Passage"],
)

doc_prompt = PromptTemplate(
    template=prompt_template,
    input_variables=['context', 'question'],
    output_parser=output_parser,
)

for document in documents:
    query = f"Select the document with name: {document}"
    docs = docsearch.as_retriever().get_relevant_documents(query)
    chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff",    prompt=doc_prompt)
    
    res = chain({"input_documents": docs, "question": diameter},    return_only_outputs=False)
    output = res['output_text'].split("\n")
    results[document] = output
    





diameter = {key:{} for key in results}
for key in results:
    for output in output_keys:
        for value in results[key]:
            if output in value:
                new_val = value.split(output)[1]
                if ':' in new_val:
                    new_val = new_val.replace(":", "")
                    diameter[key][output] = new_val
df_diameter = pd.DataFrame(diameter).drop('Source') # Not needed

df_diameter.to_csv('../data/results/graphene-oxide/diameter.csv')
df_diameter


KeyboardInterrupt: 

## Coating
Extracting adverse outcomes related to diameter from each paper:

In [ ]:
coating = """
    Adverse outcome (What was the adverse outcome?):
    Bioassay (What specific bioassay was performed?):
    Coating (What type of nanomaterial coating is reported?):
    Graphene oxide dose (What was the graphene oxide dose that produced this  outcome?):
    Graphene oxide dose units (What are the units of the graphene oxide dose that produced this  outcome?):
    Organism/Biological system (What organism or biological system was the assay performed on (include strain, if applicable)?):"""

In [ ]:
results = {}

output_parser = RegexParser(
    regex=r"\nAdverse outcome: (.*)\n\nBioassay: (.*)\nCoating type: (.*)\nGraphene oxide dose: (.*)\nGraphene oxide dose units: (.*)\nGraphene oxide dose: (.*)\nGraphene oxide dose units: (.*)\nOrganism/Biological system: (.*)\nScore: (.*)\nSource: (.*)\nPassage: (.*)",

    output_keys=["Adverse outcome","Bioassay", "Coating type", "Graphene oxide dose", "Graphene oxide dose units", "Organism/Biological system", "Score", "Source", "Passage"]
)

doc_prompt = PromptTemplate(
    template=prompt_template,
    input_variables=['context', 'question'],
    output_parser=output_parser,
)


for document in documents:
    query = f"Select the document with name: {document}"
    docs = docsearch.as_retriever().get_relevant_documents(query)
    chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff",    prompt=doc_prompt)
    
    res = chain({"input_documents": docs, "question": coating},    return_only_outputs=False)
    output = res['output_text'].split("\n")
    results[document] = output
    




coating = {key:{} for key in results}
for key in results:
    for output in output_keys:
        for value in results[key]:
            if output in value:
                new_val = value.split(output)[1]
                if ':' in new_val:
                    new_val = new_val.replace(":", "")
                    coating[key][output] = new_val
df_coating = pd.DataFrame(coating) # Not needed

df_coating.to_csv('../data/results/graphene-oxide/coating.csv')
df_coating

## Dose
Extracting adverse outcomes related to dosage from each paper:

In [ ]:
dose = """
    Adverse outcome (What was the adverse outcome?):
    Bioassay (What specific bioassay was performed?):
    Hazardous graphene oxide dose (What was the numerical value of the graphene oxide dose that produced this  outcome?):
    Hazardous graphene oxide dose units (What are the units of the graphene oxide dose that produced this  outcome?):
    Non-hazardous graphene oxide dose (What was the graphene oxide dose that that did not lead to an adverse outcome?):
    Non-hazardous graphene oxide dose units (What are the units of the graphene oxide dose that did not lead to an adverse outcome?):
    Organism/Biological system (What organism or biological system was the assay performed on (include strain, if applicable)?):
    Hazardous diameter (What is the diameter of the graphene oxide that produced this outcome?): 
    Non-hazardous diameter (What is the diameter of the graphene oxide that did not produce an outcome?): """

In [ ]:
results = {}
for document in documents:
    query = f"Select the document with name: {document}"
    docs = docsearch.as_retriever().get_relevant_documents(query)
    chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff",    prompt=doc_prompt)
    
    res = chain({"input_documents": docs, "question": dose},    return_only_outputs=False)
    output = res['output_text'].split("\n")
    results[document] = output
    
    

#TODO 
output_parser = RegexParser(
    regex=r"\nBioassay: (.*)\nMaterial diameter measurement: (.*)\nNumerical value: (.*)\nUnits: (.*)\nGraphene oxide dose: (.*)\nGraphene oxide dose units: (.*)\nOrganism/Biological system: (.*)\nScore: (.*)\nSource: (.*)\nPassage: (.*)",
    output_keys=["Adverse outcome","Bioassay", "Hazardous dose", "Hazardous dose units", "Non-hazardous dose", "Non-hazardous dose units", "Organism/Biological system", "Hazardous diameter", "Non-hazardous diameter" "score", "Source", "Passage"],
)

doc_prompt = PromptTemplate(
    template=prompt_template,
    input_variables=['context', 'question'],
    output_parser=output_parser,
)

dose = {key:{} for key in results}
for key in results:
    for output in output_keys:
        for value in results[key]:
            if output in value:
                new_val = value.split(output)[1]
                if ':' in new_val:
                    new_val = new_val.replace(":", "")
                    dose[key][output] = new_val
df_dose = pd.DataFrame(dose) # Not needed

df_dose.to_csv('../data/results/graphene-oxide/dose.csv')
df_dose

## Shape

Extracting adverse outcomes related to shape from each paper

In [ ]:
shape = """
    Adverse outcome (What was the adverse outcome?):
    Bioassay (What specific bioassay was performed?):
    Graphene oxide dose (What was the graphene oxide dose that produced this  outcome?):
    Graphene oxide dose units (What are the units of the graphene oxide dose that produced this  outcome?):
    Organism/Biological system (What organism or biological system was the assay performed on (include strain, if applicable)?):
    Hazardous shape (What is the shape of the graphene oxide that produced this outcome?):
    Hazardous diameter (What is the diameter of the graphene oxide that produced this outcome?): 
    Non-hazardous shape (What is the shape of the graphene oxide that did not produce this outcome?):
    Non-hazardous diameter (What is the diameter of the graphene oxide that did not produce this outcome?):"""

In [ ]:
results = {}
for document in documents:
    query = f"Select the document with name: {document}"
    docs = docsearch.as_retriever().get_relevant_documents(query)
    chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff",    prompt=doc_prompt)
    
    res = chain({"input_documents": docs, "question": shape},    return_only_outputs=False)
    output = res['output_text'].split("\n")
    results[document] = output
    


#TODO 
output_parser = RegexParser(
    regex=r"\nBioassay: (.*)\nMaterial diameter measurement: (.*)\nNumerical value: (.*)\nUnits: (.*)\nGraphene oxide dose: (.*)\nGraphene oxide dose units: (.*)\nOrganism/Biological system: (.*)\nScore: (.*)\nSource: (.*)\nPassage: (.*)",
    output_keys=output_keys=["Adverse outcome","Bioassay", "Hazardous dose", "Hazardous dose units", "Organism/Biological system", "Hazardous shape", "Hazardous diameter", "Non-hazardous shape", "Non-hazardous diameter" , "score", "Source", "Passage"],
)

doc_prompt = PromptTemplate(
    template=prompt_template,
    input_variables=['context', 'question'],
    output_parser=output_parser,
)

shape = {key:{} for key in results}
for key in results:
    for output in output_keys:
        for value in results[key]:
            if output in value:
                new_val = value.split(output)[1]
                if ':' in new_val:
                    new_val = new_val.replace(":", "")
                    shape[key][output] = new_val
df_shape = pd.DataFrame(shape) # Not needed

df_shape.to_csv('../data/results/graphene-oxide/shape.csv')
df_shape

## Time of exposure

Extracting adverse outcomes related to time of exposure from each paper

In [ ]:
time = """
    Adverse outcomes (The adverse outcomes?):
    Bioassays (What specific bioassays were performed?):
    Graphene oxide dose (What was the graphene oxide dose that produced this  outcome?):
    Graphene oxide dose units (What are the units of the graphene oxide dose that produced outcome i?):
    Coating (What type of nanomaterial coating is reported?):
    Organism/Biological system (What organism or biological system was the assay performed on (include strain, if applicable)?):
    Hazardous shape (What is the shape of the graphene oxide that produced this outcome?):
    Hazardous diameter (What is the diameter of the graphene oxide that produced this outcome?): 
    Non-hazardous shape (What is the shape of the graphene oxide that did not produce this outcome?):
    Non-hazardous diameter (What is the diameter of the graphene oxide that did not produce this outcome?):
    Hazardous time points (At which time points did graphene oxide produce the adverse effect?):
    Non-hazardous time points (At which time points did graphene oxide not produce the adverse effect?):"""

In [ ]:
# TODO
#results = {}
#for document in documents:
#    query = f"Select the document with name: {document}"
#    docs = docsearch.as_retriever().get_relevant_documents(query)
#    chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff",    #prompt=doc_prompt)
#    
#    res = chain({"input_documents": docs, "question": time},    #return_only_outputs=False)
#    output = res['output_text'].split("\n")
#    results[document] = output
#    
#output_parser = RegexParser(
#    regex=r"Adverse outcome: (.*)\nBioassay: (.*)\nMaterial diameter #measurement: (.*)\nNumerical value: (.*)\nUnits: (.*)\nGraphene oxide ##dose: .*)\nGraphene oxide dose units: (.*)\nOrganism/Biological system: (.##*)nScore: (.*)\nSource: (.*)\nPassage: (.*)",
#    output_keys=output_keys=["Adverse outcome","Bioassay", "Hazardous dose", #"Hazardous dose units", "Coating", "Organism/Biological system", ##"Hazardous hape", "Hazardous diameter", "Non-hazardous shape", ##"Non-hazardous iameter" , "score", "Source", "Passage"],
#)
#
#doc_prompt = PromptTemplate(
#    template=prompt_template,
#    input_variables=['context', 'question'],
#    output_parser=output_parser,
#)
#
#time = {key:{} for key in results}
#for key in results:
#    for output in output_keys:
#        for value in results[key]:
#            if output in value:
#                new_val = value.split(output)[1]
#                if ':' in new_val:
#                    new_val = new_val.replace(":", "")
#                    time[key][output] = new_val
#df_time = pd.DataFrame(time) # Not needed
#


Fix outputs

In [ ]:
def remove_parentheses(text):
    return re.sub(r'\([^)]*\)', '', text)

In [ ]:
df_diameter[df_diameter.columns.difference(['Passage'])] = df_diameter[df_diameter.columns.difference(['Passage'])].applymap(remove_parentheses)
df_diameter.to_csv('../data/results/graphene-oxide/diameter.csv')
df_diameter

In [ ]:
df_coating[df_coating.columns.difference(['Passage'])] = df_coating[df_coating.columns.difference(['Passage'])].applymap(remove_parentheses)
df_coating.to_csv('../data/results/graphene-oxide/coating.csv')
df_coating

In [ ]:
df_dose[df_dose.columns.difference(['Passage'])] = df_dose[df_dose.columns.difference(['Passage'])].applymap(remove_parentheses)
df_dose.to_csv('../data/results/graphene-oxide/dose.csv')
df_dose

In [ ]:
df_shape[df_shape.columns.difference(['Passage'])] = df_shape[df_shape.columns.difference(['Passage'])].applymap(remove_parentheses)
df_shape.to_csv('../data/results/graphene-oxide/shape.csv')
df_shape

In [ ]:
df_time[df_time.columns.difference(['Passage'])] = df_time[df_time.columns.difference(['Passage'])].applymap(remove_parentheses)
df_time.to_csv('../data/results/graphene-oxide/time.csv')
df_time